### Not in use

In [ ]:
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz
# !pip show spacy

In [1]:
# import pandas as pd
# import spacy
# from PyPDF2 import PdfReader
# from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
# from datasets import load_dataset, Dataset
# import faiss
# import numpy as np
# from tqdm import tqdm
# import torch
# import os
# import json
# from multiprocessing import Pool
# import logging
# import io


2025-03-01 10:28:12.410245: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 10:28:12.437408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-01 10:28:12.872797: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
# import pandas as pd
# import spacy
# from transformers import AutoTokenizer, AutoModel
# import faiss
# import numpy as np
# from tqdm import tqdm
# import torch
# import os
# import json
# import logging

# logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# print("Loading SciSpaCy model...")
# nlp = spacy.load("en_core_sci_sm", disable=["ner"])  # Disable NER if only tokenization needed
# nlp.enable_pipe("ner")  # Re-enable NER only when needed

# def load_csv_from_drive(csv_path):
#     print(f"Loading CSV from {csv_path}...")
    
#     if not os.path.exists(csv_path):
#         print(f"Error: CSV file {csv_path} does not exist!")
#         return None
    
#     csv_data = pd.read_csv(csv_path)
#     print(f"CSV loaded with {len(csv_data)} rows and columns: {csv_data.columns.tolist()}")

#     # Check if required columns exist
#     required_cols = ["medication_name", "symptoms", "diagnosis_SciSpaCy", "diagnosis"]
#     missing_cols = [col for col in required_cols if col not in csv_data.columns]

#     if missing_cols:
#         print(f"Warning: Missing columns: {missing_cols}")
#         # Create missing columns with NaN values
#         for col in missing_cols:
#             csv_data[col] = np.nan

#     return csv_data

# def process_csv_row(row):
#     input_text = f"Patient is taking {row['medication_name']}"
#     if pd.notna(row.get("symptoms")):
#         doc = nlp(str(row["symptoms"])[:5000])  # Limit length
#         symptoms_entities = ", ".join([ent.text for ent in doc.ents][:20])
#         input_text += f". Symptoms: {symptoms_entities}"

#     # Check which diagnosis column to use
#     if pd.notna(row.get("diagnosis_SciSpaCy")):
#         output_text = row["diagnosis_SciSpaCy"]
#     elif pd.notna(row.get("diagnosis")):
#         output_text = row["diagnosis"]
#     else:
#         return None

#     return {"input": input_text, "output": output_text}

# def prepare_fine_tune_data(csv_data):
#     csv_data = csv_data.dropna(subset=["medication_name"])  # Drop early to reduce memory
#     print(f"Processing {len(csv_data)} CSV rows...")

#     fine_tune_data = []
#     # Process rows directly instead of with Pool to avoid issues
#     for _, row in tqdm(csv_data.iterrows(), total=len(csv_data), desc="Processing CSV"):
#         result = process_csv_row(row)
#         if result is not None:
#             fine_tune_data.append(result)

#     print(f"Created {len(fine_tune_data)} training examples")

#     with open("fine_tune_data_scispacy.jsonl", "w") as f:
#         for item in fine_tune_data:
#             f.write(json.dumps(item) + "\n")

#     return fine_tune_data

# def fine_tune_biobert(fine_tune_data):
#     print("Loading BioBERT model...")
#     # Use pre-trained BERT base model instead of causal LM which was causing issues
#     model_name = "dmis-lab/biobert-base-cased-v1.1"
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModel.from_pretrained(model_name)

#     # Create dataset directly from our data
#     train_data = fine_tune_data[:int(len(fine_tune_data)*0.8)]
#     val_data = fine_tune_data[int(len(fine_tune_data)*0.8):]

#     print(f"Training on {len(train_data)} examples, validating on {len(val_data)} examples")

#     # Simple embedding-based model for matching inputs to outputs
#     class BiobertEmbeddingModel(torch.nn.Module):
#         def __init__(self, biobert_model):
#             super().__init__()
#             self.biobert = biobert_model
#             self.linear = torch.nn.Linear(768, 768)  # Assuming BERT base hidden size

#         def forward(self, input_ids, attention_mask, token_type_ids=None):
#             outputs = self.biobert(input_ids=input_ids, attention_mask=attention_mask)
#             pooled_output = outputs.last_hidden_state[:, 0]  # CLS token
#             return self.linear(pooled_output)

#     # Create a simple embedding model
#     embedding_model = BiobertEmbeddingModel(model)

#     # Save the tokenizer and model architecture
#     tokenizer.save_pretrained("./biobert_tokenizer")
#     torch.save(embedding_model.state_dict(), "./biobert_embedding_model.pt")

#     print("Saved tokenizer and model architecture (simplified training for demo)")
#     return tokenizer, embedding_model

# def create_embeddings_csv(csv_data, tokenizer, embedding_model):
#     print("Setting up embedding generation...")
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     embedding_model = embedding_model.to(device)
#     embedding_model.eval()

#     def get_biobert_embeddings(texts, batch_size=16):
#         embeddings = []
#         for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
#             batch = texts[i:i+batch_size]
#             encoded = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=512)
#             input_ids = encoded['input_ids'].to(device)
#             attention_mask = encoded['attention_mask'].to(device)

#             with torch.no_grad():
#                 outputs = embedding_model.biobert(input_ids=input_ids, attention_mask=attention_mask)
#                 # Use CLS token representation
#                 batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
#                 embeddings.extend(batch_embeddings)

#         return np.array(embeddings)

#     print("Generating medication embeddings...")
#     medication_names = csv_data["medication_name"].fillna("Unknown").tolist()
#     # Limit sample size for demonstration
#     sample_size = min(len(medication_names), 100)
#     medication_sample = medication_names[:sample_size]
    
#     # Check if medication_sample is empty
#     if len(medication_sample) == 0:
#         print("Warning: No medication names found in the CSV")
#         return np.array([]), medication_sample
    
#     medication_embeddings = get_biobert_embeddings(medication_sample)
    
#     return medication_embeddings, medication_sample

# def create_faiss_index_csv(medication_embeddings):
#     print("Creating medication FAISS index...")
    
#     if medication_embeddings.size == 0:
#         print("Warning: No embeddings to index")
#         return None
        
#     dimension = medication_embeddings.shape[1]
#     medication_index = faiss.IndexFlatL2(dimension)
#     medication_index.add(medication_embeddings)
    
#     print("FAISS medication index created successfully")
#     return medication_index

# def setup_diagnosis_system(csv_data, medication_index, tokenizer, embedding_model, medication_sample):
#     print("Setting up diagnosis system...")
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     embedding_model = embedding_model.to(device)
#     embedding_model.eval()

#     def get_embedding(text):
#         encoded = tokenizer([text], return_tensors="pt", truncation=True, padding=True, max_length=512)
#         input_ids = encoded['input_ids'].to(device)
#         attention_mask = encoded['attention_mask'].to(device)

#         with torch.no_grad():
#             outputs = embedding_model.biobert(input_ids=input_ids, attention_mask=attention_mask)
#             embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
#         return embedding

#     def diagnose_disease(prescription):
#         print(f"Analyzing prescription: {prescription}")

#         # Extract entities
#         doc = nlp(prescription[:5000])
#         entities = ", ".join([ent.text for ent in doc.ents][:20])
#         enriched_input = f"{prescription}. Entities: {entities}"
#         print(f"Extracted entities: {entities}")

#         # Get embedding
#         query_embedding = get_embedding(enriched_input)

#         # Check if medication_index exists
#         if medication_index is None:
#             return "Unable to analyze prescription: No medication data available."

#         # Search medication index
#         try:
#             D_med, I_med = medication_index.search(query_embedding, k=2)
#             if I_med.shape[0] > 0 and I_med.shape[1] > 0:
#                 similar_medications = [medication_sample[i] for i in I_med[0] if i < len(medication_sample)]
#             else:
#                 similar_medications = []
#         except Exception as e:
#             print(f"Error in medication search: {e}")
#             similar_medications = []

#         # Find potential diagnoses from similar medications
#         potential_diagnoses = []
#         for med in similar_medications:
#             matches = csv_data[csv_data['medication_name'] == med]
#             if not matches.empty:
#                 for _, row in matches.iterrows():
#                     if pd.notna(row.get('diagnosis')):
#                         potential_diagnoses.append(row['diagnosis'])
#                     elif pd.notna(row.get('diagnosis_SciSpaCy')):
#                         potential_diagnoses.append(row['diagnosis_SciSpaCy'])

#         # Build diagnosis
#         if potential_diagnoses:
#             diagnosis = f"Based on the prescription '{prescription}', the most likely diagnosis is: {potential_diagnoses[0]}"
#             if len(potential_diagnoses) > 1:
#                 diagnosis += f"\nOther possible diagnoses: {', '.join(potential_diagnoses[1:])}"
#         else:
#             diagnosis = f"Unable to determine a specific diagnosis for '{prescription}' based on available data."

#         return diagnosis

#     print("Diagnosis system ready")
#     return diagnose_disease

# # Main execution function
# def main():
#     print("==== Medical NLP System with SciSpaCy and BioBERT (No PDFs) ====")

#     # Specify path to CSV in Google Drive
#     csv_path = 'merge_demo_amos_v3.csv'  # Update this path

#     # Load CSV data from Google Drive
#     csv_data = load_csv_from_drive(csv_path)
    
#     if csv_data is None:
#         print("Error loading CSV data. Exiting.")
#         return

#     # Prepare training data
#     fine_tune_data = prepare_fine_tune_data(csv_data)

#     # Setup tokenizer and model
#     tokenizer, embedding_model = fine_tune_biobert(fine_tune_data)

#     # Generate embeddings for medication data
#     medication_embeddings, medication_sample = create_embeddings_csv(csv_data, tokenizer, embedding_model)

#     # Create FAISS index for medications
#     medication_index = create_faiss_index_csv(medication_embeddings)

#     # Setup diagnosis function
#     diagnose_disease = setup_diagnosis_system(
#         csv_data, medication_index, tokenizer, embedding_model, medication_sample
#     )

#     # Example diagnosis
#     print("\n--- Sample Diagnosis ---")
#     prescription = "Patient is taking Metformin 500mg daily"
#     result = diagnose_disease(prescription)
#     print(f"\nSample Diagnosis Result:\n{result}")

#     print("\nSystem is ready for use. You can diagnose diseases by providing prescriptions.")

#     # Interactive mode
#     while True:
#         user_input = input("\nEnter prescription (or 'exit' to quit): ")
#         if user_input.lower() == 'exit':
#             break

#         diagnosis = diagnose_disease(user_input)
#         print(f"\nDiagnosis:\n{diagnosis}")

# if __name__ == "__main__":
#     main()

Loading SciSpaCy model...
==== Medical NLP System with SciSpaCy and BioBERT (No PDFs) ====
Loading CSV from merge_demo_amos_v3.csv...
CSV loaded with 7541 rows and columns: ['Unnamed: 0', 'disease_name', 'description', 'symptoms', 'causes', 'source', 'table_name', 'treatment', 'url', 'diagnosis', 'medication_name', 'medication_url', 'dosage_url', 'prognosis', 'research', 'complications', 'faq', 'transmission', 'disease_name_standardized', 'source_standardized', 'symptoms_SciSpaCy', 'diagnosis_SciSpaCy', 'medication_SciSpaCy', 'treatment_SciSpaCy', 'treatment_BioBERT']
Processing 2314 CSV rows...


Processing CSV: 100%|█████████████████████████████████████████████████████████████| 2314/2314 [00:00<00:00, 8015.43it/s]


Created 1 training examples
Loading BioBERT model...
Training on 0 examples, validating on 1 examples
Saved tokenizer and model architecture (simplified training for demo)
Setting up embedding generation...
Generating medication embeddings...


Generating embeddings: 100%|██████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 32.19it/s]


Creating medication FAISS index...
FAISS medication index created successfully
Setting up diagnosis system...
Diagnosis system ready

--- Sample Diagnosis ---
Analyzing prescription: Patient is taking Metformin 500mg daily
Extracted entities: Patient, Metformin 500mg, daily

Sample Diagnosis Result:
Unable to determine a specific diagnosis for 'Patient is taking Metformin 500mg daily' based on available data.

System is ready for use. You can diagnose diseases by providing prescriptions.



Enter prescription (or 'exit' to quit):  conditions that cause afib such as thyroid disease will be treated you may also need any of the following heart medicines help control your heart rate or rhythm you may need more than medicine to treat your symptoms antiplatelet or blood thinner medicines help prevent blood clots and stroke cardioversion is a procedure to return your heart rate and rhythm to normal it can be done using medicines or electric shock afib ablation is a procedure that uses energy to burn a small area of heart tissue this creates scar tissue and prevents electrical signals that cause afib you may need this procedure more than time ask for more information on afib ablation a pacemaker may be inserted into your heart a pacemaker is a device that controls your heartbeat a pacemaker may be inserted during an ablation procedure or surgery ask your healthcare provider for more information on pacemakers surgery may be needed if other procedures do not work during surgery yo

Analyzing prescription: conditions that cause afib such as thyroid disease will be treated you may also need any of the following heart medicines help control your heart rate or rhythm you may need more than medicine to treat your symptoms antiplatelet or blood thinner medicines help prevent blood clots and stroke cardioversion is a procedure to return your heart rate and rhythm to normal it can be done using medicines or electric shock afib ablation is a procedure that uses energy to burn a small area of heart tissue this creates scar tissue and prevents electrical signals that cause afib you may need this procedure more than time ask for more information on afib ablation a pacemaker may be inserted into your heart a pacemaker is a device that controls your heartbeat a pacemaker may be inserted during an ablation procedure or surgery ask your healthcare provider for more information on pacemakers surgery may be needed if other procedures do not work during surgery your healthcare prov

KeyboardInterrupt: Interrupted by user

### Model

In [2]:
# Import libraries
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import gc
import json

# Define file paths
DATA_PATH = "processed_diseases-583.csv"
MODEL_SAVE_PATH = "model"

# PyTorch optimizations
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

# Load CSV
try:
    df = pd.read_csv(
        DATA_PATH,
        usecols=["Disease", "Treatment", "Symptoms"],
        engine="pyarrow",
        dtype_backend="numpy_nullable",
        dtype={"Disease": "string", "Treatment": "string", "Symptoms": "string"}
    ).rename(columns={"Disease": "disease_name", "Treatment": "treatment", "Symptoms": "symptoms"})
    print(f"Loaded CSV with {len(df)} rows")
except Exception as e:
    raise ValueError(f"Error loading CSV: {e}")

# Verify columns
required_cols = ["symptoms", "disease_name", "treatment"]
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"Missing columns: {[col for col in required_cols if col not in df.columns]}")

# Clean NaN values and prepare data
df["symptoms"] = df["symptoms"].fillna("").str.strip()
df["disease_name"] = df["disease_name"].fillna("").str.strip()
df["treatment"] = df["treatment"].fillna("").str.strip()

# Remove rows with empty symptoms or both empty disease and treatment
df = df[(df["symptoms"] != "") & ((df["disease_name"] != "") | (df["treatment"] != ""))]
print(f"Dataset after cleaning: {len(df)} rows")

# Check for data statistics
avg_symptom_len = df["symptoms"].str.len().mean()
avg_disease_len = df["disease_name"].str.len().mean()
avg_treatment_len = df["treatment"].str.len().mean()
print(f"Average lengths - Symptoms: {avg_symptom_len:.1f}, Disease: {avg_disease_len:.1f}, Treatment: {avg_treatment_len:.1f}")

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For a generation task, we'll use a seq2seq model (e.g., T5 or BART)
try:
    # Using a biomedical seq2seq model suitable for generation tasks
    model_name = "GanjinZero/biobart-v2-base"  # Biomedical BART model for generation
    
    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_name,
        torch_dtype=torch.float32
    ).to(device)
    print(f"Initialized {model_name}")
except RuntimeError as e:
    print(f"Memory error initializing model: {e}")
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    raise

# Custom Dataset for seq2seq task
class MedicalGenerationDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_length=512, max_target_length=128, task_type="both"):
        self.df = df
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length
        self.task_type = task_type  # 'diagnosis', 'treatment', or 'both'

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        symptoms = row["symptoms"].strip()
        
        # Prepare input based on task
        if self.task_type == "diagnosis":
            input_text = f"Generate a diagnosis based on these symptoms: {symptoms}"
            target_text = row["disease_name"].strip()
        elif self.task_type == "treatment":
            input_text = f"Generate a treatment plan based on these symptoms: {symptoms}"
            target_text = row["treatment"].strip()
        else:  # both
            disease = row["disease_name"].strip()
            input_text = f"Generate a diagnosis and treatment plan based on these symptoms: {symptoms}"
            treatment = row["treatment"].strip()
            target_text = f"Diagnosis: {disease} Treatment: {treatment}"
        
        # Skip examples with empty targets
        if not target_text.strip():
            target_text = "No information available"
        
        # Tokenize inputs and targets
        model_inputs = self.tokenizer(
            input_text,
            padding="max_length",
            truncation=True,
            max_length=self.max_input_length,
            return_tensors="pt"
        )
        
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                target_text,
                padding="max_length",
                truncation=True,
                max_length=self.max_target_length,
                return_tensors="pt"
            )
        
        # Remove batch dimension
        input_ids = model_inputs["input_ids"].squeeze()
        attention_mask = model_inputs["attention_mask"].squeeze()
        labels = labels["input_ids"].squeeze()
        
        # Replace padding token id with -100 for loss calculation
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# Choose task type - can be "diagnosis", "treatment", or "both"
TASK_TYPE = "both"

# Prepare datasets
train_df, eval_df = train_test_split(
    df, test_size=0.2, random_state=42
)
print(f"Train size: {len(train_df)}, Eval size: {len(eval_df)}")

train_dataset = MedicalGenerationDataset(train_df, tokenizer, task_type=TASK_TYPE)
eval_dataset = MedicalGenerationDataset(eval_df, tokenizer, task_type=TASK_TYPE)

# Custom metrics for generation evaluation
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    
    # Replace -100 with pad token id
    pred_mask = labels_ids != -100
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    
    # Ensure pred_ids is a list of lists of integers
    # This fixes the overflow error
    if hasattr(pred_ids, "cpu"):
        pred_ids = pred_ids.cpu().numpy()
    
    # Ensure all values are within valid integer range
    # This is a more robust approach for handling the token IDs
    
    # First, log the range of values
    print(f"Raw pred_ids shape: {pred_ids.shape}, type: {type(pred_ids)}")
    if hasattr(pred_ids, "size"):  # Check if it's a numpy array
        print(f"Max raw pred_id: {np.max(pred_ids)}, Min raw pred_id: {np.min(pred_ids)}")
    
    # Make sure we have integer values
    if np.issubdtype(pred_ids.dtype, np.floating):
        print("Converting floating point predictions to integers")
        pred_ids = np.rint(pred_ids).astype(np.int64)
    
    # Use a safer approach: process batch item by item
    pred_str = []
    for i in range(len(pred_ids)):
        try:
            # Get a single item from the batch
            item_ids = pred_ids[i].tolist() if hasattr(pred_ids[i], "tolist") else list(pred_ids[i])
            
            # Ensure all IDs are valid Python integers
            item_ids = [int(id) for id in item_ids if id != -100]
            
            # Filter out any potentially problematic token IDs (very large values)
            item_ids = [id for id in item_ids if id < tokenizer.vocab_size]
            
            # Decode a single item
            decoded = tokenizer.decode(item_ids, skip_special_tokens=True)
            pred_str.append(decoded)
        except Exception as e:
            print(f"Error decoding item {i}: {e}")
            # Fallback to an empty string if decoding fails
            pred_str.append("")
    
    # Process labels similarly
    label_str = []
    for i in range(len(labels_ids)):
        try:
            # Get a single item
            item_ids = labels_ids[i].tolist() if hasattr(labels_ids[i], "tolist") else list(labels_ids[i])
            
            # Ensure all IDs are valid Python integers
            item_ids = [int(id) for id in item_ids if id != -100]
            
            # Decode a single item
            decoded = tokenizer.decode(item_ids, skip_special_tokens=True)
            label_str.append(decoded)
        except Exception as e:
            print(f"Error decoding label {i}: {e}")
            label_str.append("")
    
    # Calculate ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}
    
    # Calculate BLEU and ROUGE for each prediction
    bleu_scores = []
    
    for pred, label in zip(pred_str, label_str):
        # Skip empty predictions or labels
        if not pred.strip() or not label.strip():
            continue
            
        # BLEU score with smoothing
        try:
            pred_tokens = pred.split()
            label_tokens = label.split()
            smoothie = SmoothingFunction().method1
            bleu = sentence_bleu([label_tokens], pred_tokens, smoothing_function=smoothie)
            bleu_scores.append(bleu)
        except Exception as e:
            print(f"Error calculating BLEU: {e}")
            
        # ROUGE scores
        try:
            scores = scorer.score(label, pred)
            rouge_scores["rouge1"] += scores["rouge1"].fmeasure
            rouge_scores["rouge2"] += scores["rouge2"].fmeasure
            rouge_scores["rougeL"] += scores["rougeL"].fmeasure
        except Exception as e:
            print(f"Error calculating ROUGE: {e}")
    
    # Calculate averages
    n_samples = len(pred_str)
    if n_samples > 0:
        rouge_scores["rouge1"] /= n_samples
        rouge_scores["rouge2"] /= n_samples
        rouge_scores["rougeL"] /= n_samples
    
    # Average BLEU score
    avg_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    
    # Return all metrics
    metrics = {
        "bleu": avg_bleu,
        "rouge1": rouge_scores["rouge1"],
        "rouge2": rouge_scores["rouge2"],
        "rougeL": rouge_scores["rougeL"]
    }
    
    return metrics

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=os.path.join(MODEL_SAVE_PATH, f"results_{TASK_TYPE}"),
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir=os.path.join(MODEL_SAVE_PATH, "logs"),
    logging_steps=100,
    save_total_limit=3,
    predict_with_generate=True,
    generation_max_length=128,
    generation_num_beams=4,
    fp16=False,  # Avoid FP16 issues
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="rouge2",
    greater_is_better=True,
    report_to="none"
)

# Initialize trainer
try:
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    
    print("Starting training...")
    trainer.train()
    
    eval_results = trainer.evaluate()
    print("Evaluation Results:", eval_results)
except RuntimeError as e:
    print(f"Error during training: {e}")
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    raise

# Save model
try:
    model_save_path = os.path.join(MODEL_SAVE_PATH, f"medical_generation_{TASK_TYPE}")
    os.makedirs(model_save_path, exist_ok=True)
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    
    # Save training configuration
    config = {
        "task_type": TASK_TYPE,
        "model_name": model_name,
        "dataset_size": len(df),
        "train_size": len(train_df),
        "eval_size": len(eval_df),
        "metrics": eval_results
    }
    
    with open(os.path.join(model_save_path, "training_config.json"), "w") as f:
        json.dump(config, f, indent=2)
    
    print(f"Model and configuration saved to {model_save_path}")
except Exception as e:
    print(f"Error saving model: {e}")

# Test the model with sample inputs
def generate_medical_output(symptoms, task_type=TASK_TYPE):
    # Prepare input based on task
    if task_type == "diagnosis":
        input_text = f"Generate a diagnosis based on these symptoms: {symptoms}"
    elif task_type == "treatment":
        input_text = f"Generate a treatment plan based on these symptoms: {symptoms}"
    else:  # both
        input_text = f"Generate a diagnosis and treatment plan based on these symptoms: {symptoms}"
    
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    
    # Generate output
    try:
        outputs = model.generate(
            inputs["input_ids"],
            max_length=128,
            num_beams=4,
            early_stopping=True
        )
        
        # Handle token ID decoding in a more robust way
        try:
            # First attempt: direct decoding if possible
            decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        except Exception as e:
            print(f"Error in direct decoding: {e}")
            
            # Second attempt: Convert to Python list and filter
            try:
                # Convert to CPU and numpy if needed
                if hasattr(outputs, "cpu"):
                    outputs_list = outputs[0].cpu().tolist()
                else:
                    outputs_list = outputs[0].tolist() if hasattr(outputs[0], "tolist") else list(outputs[0])
                
                # Filter out any potentially problematic token IDs
                outputs_list = [int(id) for id in outputs_list if id < tokenizer.vocab_size]
                
                # Try decoding the filtered list
                decoded = tokenizer.decode(outputs_list, skip_special_tokens=True)
                print("Successfully decoded after filtering")
            except Exception as e2:
                print(f"Error in fallback decoding: {e2}")
                decoded = "Error: Could not decode model output"
            
        return decoded
    except Exception as e:
        print(f"Error during generation: {e}")
        return "Error generating output"

# Sample test with a few examples from the test set
if len(eval_df) > 0:
    print("\nTesting model with sample inputs:")
    
    # Select a few random samples from eval set
    sample_indices = np.random.choice(len(eval_df), min(3, len(eval_df)), replace=False)
    
    for idx in sample_indices:
        sample = eval_df.iloc[idx]
        symptoms = sample["symptoms"]
        actual_disease = sample["disease_name"]
        actual_treatment = sample["treatment"]
        
        print(f"\nInput symptoms: {symptoms}")
        print(f"Actual disease: {actual_disease}")
        print(f"Actual treatment: {actual_treatment}")
        
        # Generate output
        generated_output = generate_medical_output(symptoms)
        print(f"Generated output: {generated_output}")

# Clean up
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

2025-03-08 15:02:49.209341: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 15:02:49.236891: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-08 15:02:49.834628: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loaded CSV with 1318 rows
Dataset after cleaning: 1318 rows
Average lengths - Symptoms: 49.2, Disease: 20.7, Treatment: 62.1
Using device: cuda
Initialized GanjinZero/biobart-v2-base
Train size: 1054, Eval size: 264
Starting training...


/home/saurabh/.local/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/saurabh/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel
1,No log,1.378041,0.044186,0.346518,0.090076,0.333695
2,2.091000,1.203485,0.067337,0.390379,0.129102,0.374719
3,2.091000,1.146376,0.092835,0.406607,0.154802,0.390324
4,1.143100,1.106375,0.101020,0.421353,0.169578,0.403320
5,0.894000,1.091572,0.105167,0.425665,0.173274,0.405111


Raw pred_ids shape: (264, 39), type: <class 'numpy.ndarray'>
Max raw pred_id: 85258, Min raw pred_id: -100


...

/home/saurabh/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/saurabh/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Raw pred_ids shape: (264, 44), type: <class 'numpy.ndarray'>
Max raw pred_id: 85328, Min raw pred_id: -100


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
/home/saurabh/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Raw pred_ids shape: (264, 44), type: <class 'numpy.ndarray'>
Max raw pred_id: 85328, Min raw pred_id: -100
Evaluation Results: {'eval_loss': 1.0915718078613281, 'eval_bleu': 0.10516674331808576, 'eval_rouge1': 0.42566452233779095, 'eval_rouge2': 0.1732742979633933, 'eval_rougeL': 0.40511128643722766, 'eval_runtime': 15.5673, 'eval_samples_per_second': 16.959, 'eval_steps_per_second': 2.12, 'epoch': 5.0}
Model and configuration saved to model/medical_generation_both

Testing model with sample inputs:

Input symptoms: Wrinkled abdomen, urinary issues, undescended testes.
Actual disease: PruneBelly Syndrome
Actual treatment: Surgery (reconstruction), antibiotics (e.g., amoxicillin), support.
Generated output: Diagnosis: Klinefelter Syndrome (Entry 2) Treatment: Testosterone gel (0.5% cream), surgery if small.

Input symptoms: Heartburn, regurgitation, chest pain, dysphagia.
Actual disease: Hiatal Hernia
Actual treatment: Omeprazole (2040 mg daily), surgery (fundoplication), lifestyle chan

433

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Define model path and device
MODEL_SAVE_PATH = "model/medical_generation_both/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_SAVE_PATH).to(device)

# Function to generate diagnosis and treatment
def generate_medical_advice(symptoms, max_length=128):
    """Generates diagnosis and treatment based on symptoms."""
    
    # Prepare input
    input_text = f"Generate a diagnosis and treatment plan based on these symptoms: {symptoms}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    
    # Generate output
    with torch.no_grad():
        output_tokens = model.generate(**inputs, max_length=max_length, num_beams=5, early_stopping=True)
    
    # Decode output
    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
    return generated_text

# Example usage
if __name__ == "__main__":
    test_symptoms = "Fever, cough, difficulty breathing"
    result = generate_medical_advice(test_symptoms)
    print("Generated Diagnosis & Treatment:", result)


Generated Diagnosis & Treatment: Diagnosis: COVID19 (Coronavirus Disease 2019) Treatment: Oseltamivir (75 mg twice daily), oxygen, rest.
